02で前処理をしたデータの読み込みと03で学習したモデルを使って推論を行うためのnotebookです。  

## 必要なライブラリのimport

In [2]:
%pip install pandas
%pip install sqlalchemy
%pip install scikit-learn
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pickle
# import xgboost as xgb

In [2]:
# 予測したいデータの読み込み
test = pd.read_csv('../data/processed/processed20240626_history_test.csv')

## 保存されたモデルを使って予測を行う

In [3]:
n_folds = 5
predictions = np.zeros(len(test))
features = [c for c in test.columns if c not in ['card_id', 'first_active_month']]

for fold_ in range(n_folds):
    # モデルを読み込む
    with open(f'C:/Users/momoka.miyaguchi.kd/OneDrive - AMBL株式会社/Python研修/kaggle/AnalyticalPipeline/src/models/model_fold_{fold_}.pkl', 'rb') as f:
        clf = pickle.load(f)

    # xgb用
    # dtest = xgb.DMatrix(test[features])
    # テストデータをDMatrix形式に変換する
    # fold_predictions = clf.predict(dtest)

    # テストデータをDMatrix形式に変換する
    # モデルで予測を行う
    fold_predictions = clf.predict(test[features])

    # 予測結果を足し合わせる
    predictions += fold_predictions

# 平均を取る
predictions /= n_folds

# RMSEを計算する（テストデータに対して真の値がある場合）
# y_testが存在する場合に限り
# if y_test in locals():
#     score = mean_squared_error(y_test, predictions, squared=False)
#     print(f'RMSE={score}')

In [4]:
# 予測結果を出力する
print(predictions)

[-3.80226671 -0.33160333 -1.33314998 ...  0.88109584 -5.00728222
  0.1473556 ]


## 提出用ファイルの作成

In [5]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submit_0626.csv", index=False)

In [11]:
df = pd.read_csv('submit_0621.csv')

# card_id列で重複をチェック
duplicate_cards = df[df.duplicated('card_id')]
print("Duplicate card_ids:\n", duplicate_cards['card_id'].unique())

Duplicate card_ids:
 ['C_ID_0ab67a22ab' 'C_ID_2b5e3df5c2' 'C_ID_a1b3c75277' ...
 'C_ID_21d56d950c' 'C_ID_6c46fc5a9d' 'C_ID_87e7979a5f']


In [14]:
# 重複を削除
df = df.drop_duplicates('card_id', keep='first')

In [15]:
# 新しいファイルに保存する
new_filename = 'submit_re.csv'  # 保存するファイル名を指定します
df.to_csv(new_filename, index=False)  # CSVファイルとして保存します（index列を保存しない設定）

print(f"Processed data saved to {new_filename}")

Processed data saved to submit_re.csv
